In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import math

# **Input Embedding**

In [ ]:
class InputEmbeddings(nn.Module):
    
    def __init__(self, d_model : int, vocab_size : int):
        super().__init__() 
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)
        
    def forward(self, x):
        return self.embedding(x) * self(d_model)**-0.5

# **Positional Embedding**

In [ ]:
class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model : int, seq_len : int, dropout : float) -> None:
        super().__init__()
        self.d_model = d_model 
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        
        #matrix creation of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        position = torch.arange(0, seq_len, dtype = torch.float).unsqueeze(1) 
        div_term = torch.exp(torch.arange(0,d_model,2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        
        self.register_buffer('pe', pe)
        
    def forward(self,x):
        x = x + (self.pe[:, :x_shape[1], :]).requires_grad_(False)
        return self.dropout(x)
        

# **Encoder Block**

# **Layer Normalization**

In [ ]:
class LayerNormalization(nn.Module):
    
    def __init__(self, eps: float = 10** - 6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(1))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        mean = x.mean(dim = -1, keepdim = True)
        std = x.std(dim = -1, keepdim = True)
        return self.alpha * (x- mean) / (std + self.eps) + self.bias

# **Feed forward**

In [ ]:
class FeedForward(nn.Module):
    
    def __init__(self,d_model : int,  d_ff: int, dropout : float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))
    

# **Multihead Attention**

In [ ]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self, d_model : int, h : int, dropout : float) -> None:
        super().__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        self.h = h
        assert d_model % h == 0
        
        self.d_k = d_model // h
        
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        
        self.w_o = nn.Linear(d_model, d_model)
    
    @staticmethod
    def selfattention(query, key, value, mask, dropout : nn.Dropout):
        d_k = query.shape[-1]
        attention_scores = (query @ key.transpose(-2, -1)) // math.sqrt(d_k)
        if mask is not None:
            attention_scores.masked_fill_(mask == 0, float('-inf'))
        attention_scores = attention_scores.softmax(dim = -1) 
        
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        
        return (attention_scores @ value), attention_scores
        
    def forward(self, q,k,v,mask):
        query = self.w_q(q) #batch, seq_len, d_model -> batch, seq_len, d_model
        key = self.w_k(k)   #batch, seq_len, d_model -> batch, seq_len, d_model
        value = self.w_v(v) #batch, seq_len, d_model -> batch, seq_len, d_model
            
            
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1,2) # batch, seq_len, d_model -> batch, seq_len, h, d_k -> batch, h, seq_len, d_k 
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1,2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1,2)
        
        x.self.attention_scores = MultiHeadAttention.selfattention(query, key, value, mask, self.dropout)
        
        # batch, h, seq_len, d_k -> batch, seq_len, h, d_k -> batch, seq_len, d_model
        x = x.transpose(1,2).contiguous().view(x.shape[0], -1, self.h * self.d_k)
        
        # batch, seq_len, d_model ->  batch, seq_len, d_model
        return self.w_o(x)
        
        

#  **Residual Connections**

In [ ]:
class ResidualConnection(nn.Module):
    
    def __init__(self, features: int, dropout: float):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization(features)
        
        
        def forward(self, x, sublayer):
            #return x + self.norm(sublayer(self.dropout(x)))
            return x + self.dropout(sublayer(self.norm(x)))

# **Block Completion**

In [ ]:
class EncoderBlock(nn.Module):
    
    def __init__(self,features: int, self_attention_block : MultiHeadAttention, feed_forward_block : FeedForward, dropout : float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([
            ResidualConnection(dropout) for _ in range(2)
        ])
        
    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x,x,x,src_mask))
        x = self.feed_forward_block[1](x, self.feed_forward_block)
        return x
    

In [ ]:
class Encoder(nn.Module):
    
    def __init__(self, layers : nn.ModuleList)->None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()
        
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)    
        return self.norm(x)

# **Decoder** 

In [ ]:
class DecoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttention, cross_attention_block: MultiHeadAttention, feed_forward_block: FeedForward, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

# **Decoder Block**

In [ ]:
class Decoder(nn.Module):
    
    def __init__(self, layers : nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()
        
    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)
        
        

# **Linear Layer** 
**(Output Layer with Softmax Application)**

In [ ]:
class ProjectionLayer(nn.Module):
    
    def __init__(self, d_model : int, vocab_size: int) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)
        
    def forward(self, x):
        #batch, seq_len, d_model -> batch, seq_len, vocab_size
        return torch.log_softmax(self.proj(x), dim = -1)
    
        

# **Transformer Architecture**

In [ ]:
class Transformer(nn.Module):
    def __init__(self, encoder : Encoder, decoder : Decoder, src_embed: InputEmbeddings, tgt_embed : InputEmbeddings, src_pos : PositionalEncoding, tgt_pos : PositionalEncoding, projection_layer : ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer
    
    def encode(self, src, src_mask):
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)
    
    def decode(self, encoder_output, src_mask, tgt, tgt_mask):
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)
    
    def project(self, x):
        return self.projection_layer(x)


# **Combining all the functions**
> Build Transformer function 

Code with this block can be used for any task that requires a transformer architecture, not just the language conversion task that i will do in my notebook. **Naming that i have used in the next block is specified to be used in my task, that is hindi to english translation, and that will have to be changed when using it for a specified task.** 

In [ ]:
def build_transformer(src_vocab_size : int, tgt_vocab_size : int, src_seq_len : int, tgt_seq_len : int, d_model : int = 512, N : int = 6, h: int = 8, dropout : float = 0.1, d_ff : int = 2048):
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)
    
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)
    
    # ENCODER BLOCK
    
    encoder_blocks = []
    
    
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttention(d_model, h, dropout)
        feed_forward_block = FeedForward(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)
     
    # DECODER BLOCK
    
    decoder_blocks = []
    
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttention(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttention(d_model, h, dropout)
        feed_forward_block = FeedForward(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)
        
    encoder = Encoder(nn.ModuleList(encoder_block))
    decoder = Decoder(nn.ModuleList(decoder_block))
    
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)
    
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)
    
    # Xavier Uniform
    # Parameters are initialized by xavier uniform algorithm which makes the training faster by not initializing parameters randomly.

    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
            
    return transformer

# **Tokenizer** 
> **word level** 

In [ ]:
pip install tokenizers

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path

In [ ]:
def get_all_sentences(ds, lang):
    for item in ds:
        yield item['translation'][lang]

In [ ]:
def get_or_build_tokenizer(config, ds, lang):
    tokenizer_path = Path(config['tokenizer_file'].format(lang))
    if not Path.exists(tokenizer_path):
        tokenizer = Tokenizer(WordLevel(unk_token = '[UNK]'))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens = ["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency= 2)
        tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer = trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer


In [ ]:
import os

tokenizer_folder_path = '/kaggle/working/tokenizer'
model_folder_path = '/kaggle/working/weights'

os.makedirs(tokenizer_folder_path, exist_ok=True)
os.makedirs(model_folder_path, exist_ok=True)

print(f"Folder '{tokenizer_folder_path}' created successfully!")


In [ ]:
def get_config():
    config = {
        "datasource" : 'cfilt/iitb-english-hindi',
        "num_epochs": 20,
        "lr": 10**-4,
        "lang_src": "en",  
        "lang_tgt": "hi",
        "seq_len": 2200,
        "batch_size": 4,
        "d_model": 512,
        "model_folder": "/kaggle/working/weights",
        "model_basename": "/kaggle/working/weights/tmodel_",
        "tokenizer_file": "/kaggle/working/tokenizer/tokenizer_{}.json",
        "preload": None,
        "experiment_name": "/kaggle/working/runs/tmodel"
    }
    
    return config

config = get_config()

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split
from datasets import load_dataset

def get_ds(config):
    
    ds_train = load_dataset('cfilt/iitb-english-hindi', 'default', split='train')
    
    return ds_train

ds_train = get_ds(config)

In [ ]:
from pathlib import Path

def get_weights_file_path(config, epoch: str):
    model_folder = config['model_folder']
    model_basename = config['model_basename']
    model_filename = f"{model_basename}{epoch}.pt"
    return str(Path('_') / model_folder / model_filename)

In [ ]:
def get_weights_file_path(config, epoch: str):
    model_folder = config 

In [ ]:
tokenizer_src = get_or_build_tokenizer(config, ds_train, config['lang_src'])
tokenizer_tgt = get_or_build_tokenizer(config, ds_train, config['lang_tgt'])

In [ ]:
from torch.utils.data import random_split

train_ds_size = int(0.9* len(ds_train))
val_ds_size = len(ds_train) - train_ds_size
train_ds, val_ds = random_split(ds_train, [train_ds_size, val_ds_size])

# **Dataset**

In [ ]:
def casual_mask(size):
    mask = torch.triu(torch.ones(1,size,size, diagonal = 1).type(torch.int))
    return mask == 0

In [ ]:
from typing import Any

class BilingualDataset(Dataset):
    def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len) -> None:
        super().__init__()
        
        self.ds = ds
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang
        self.seq_len = seq_len
        
        self.sos_token = torch.tensor([tokenizer_src.token_to_id(('[SOS]'))], dtype = torch.int64)
        self.eos_token = torch.tensor([tokenizer_src.token_to_id(('[EOS]'))], dtype = torch.int64)
        self.pad_token = torch.tensor([tokenizer_src.token_to_id(('[PAD]'))], dtype = torch.int64)
        
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, index : Any) -> Any:
        src_target_pair = self.ds[index]
        src_text = src_target_pair['translation'][self.src_lang]
        tgt_text = src_target_pair['translation'][self.tgt_lang]
        
        enc_input_tokens = self.tokenizer_src.encode(src_text).ids
        dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids
        
        enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2
        dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1
        
        if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0 :
            raise ValueError('Sentence is too long')
        
        encoder_input = torch.cat(
        [
            self.sos_token, 
            torch.tensor(enc_input_tokens, dtype = torch.int64),
            self.eos_token,
            torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype = torch.int64)
        ])
        
        decoder_input = torch.cat(
        [
            self.sos_token,
            torch.tensor(dec_input_tokens, dtype = torch.int64),
            torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype = torch.int64)
        ])
        
        label = torch.cat(
        [
            torch.tensor(dec_input_tokens, dtype = torch.int64),
            self.eos_token,
            torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype = torch.int64)
        ])
        
        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len
        
        return {
            "encoder_input" : encoder_input,
            "decoder_input" : decoder_input,
            "encoder_mask"  : (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(),
            "decoder_mask"  : (decoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int() & casual_mask(decoder_input.size(0)),
            "label"         : label,
            "src_text"      : src_text,
            "tgt_text"      : tgt_text
            
        }

In [ ]:
print(type(ds_train))  
print(type(train_ds))
print(type(val_ds))

In [ ]:
train_ds = BilingualDataset(train_ds, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
val_ds = BilingualDataset(val_ds, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])

In [ ]:
print(f"Size of ds_train: {len(ds_train)}")
print(f"Size of train_ds: {len(train_ds)}")
print(f"Size of val_ds: {len(val_ds)}")

In [ ]:
max_len_src = 0

max_len_tgt = 0

for item in ds_train:
    src_ids = tokenizer_src.encode(item ['translation'][config['lang_src']]).ids
    tgt_ids = tokenizer_tgt.encode(item ['translation'][config['lang_tgt']]).ids
    
    max_len_src = max(max_len_src, len(src_ids))
    max_len_tgt = max(max_len_tgt, len(tgt_ids))
    
print(f'Max length of source sentence: {max_len_src}')
print(f'Max length of target sentence: {max_len_tgt}')

# **DataLoaders**

In [ ]:
train_dataloader = DataLoader(train_ds, batch_size = config['batch_size'], shuffle = True)
val_dataloader = DataLoader(val_ds, batch_size = 1, shuffle = True)

In [ ]:
print(type(train_dataloader))  
print(type(val_dataloader))

In [ ]:
print(f"Size of train_dataloader: {len(train_dataloader)}")
print(f"Size of val_dataloader: {len(val_dataloader)}")

In [ ]:
def get_model(config, vocab_src_len, vocab_tgt_len):
    model = build_transformer(vocab_src_len, vocab_tgt_len, config['seq_len'],config['seq_len'], config['d_model'])
    
    return model

In [ ]:
import tensorboard
from tqdm import tqdm
import warnings

# **Training Loop** 

In [ ]:
def train_model(config):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device {device}')
    
    Path(config['model_folder']).mkdir(parents=True, exist_ok = True)
    
    model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)
    
    writer = SummaryWriter(config['experiment_name'])
    
    optimizer = torch.optim.Adam(model.parameters(), lr = config['lr'], eps = 1e-9)
    
    initial_epoch = 0
    global_step = 0
    
    if config['preload']:
        model_filename = get_weights_file_path(config, config['preload'])
        print(f'Preloading model {model_filename}')
        state = torch.load(model_filename)
        initial_epoch = state['epoch'] + 1
        optimizer.load_state_dict(state['optimizer_state_dict'])
        global_step = state['global_step']
        
    loss_fn = nn.CrossEntropyLoss(ignore_index = tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)
    
    for epoch in range(initial_epoch, config['num_epochs']):
        model.train()
        batch_iterator = tqdm(train_dataloader, desc = f'Processing epoch {epoch :02d}')
        for batch in batch_iterator:
            
            encoder_input = batch['encoder_input'].to(device) 
            decoder_input = batch['decoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)
            decoder_mask = batch['decoder_mask'].to(device)
            
            encoder_output = model.encode(encoder_input, encoder_mask) #(batch, seq_len, d_model)
            decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask) #(batch, seq_len, d_model)
            proj_output = model.project(decoder_output) #(batch, seq_len, tgt_vocab_size)
            
            label = batch['label'].to(device) #(batch, seq_len)
            
            loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1)) #(batch, seq_len, tgt_vocab_size) -> (batch * seq_len, tgt_vocab_size)
            batch_iterator.set_postfix({f"loss": f"{loss.item():6.3f}"})
            
            writer.add_scaler('train loss', loss.item(), global_step)
            writer.flush()
            
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
            
            global_step +=1
            
            model_filename = get_weights_file_path(config, f'{epoch:02d}')
            torch.save({
                'epoch' : epoch,
                'model_state_dict' : model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'global_step' : global_step,
            }, model_filename)
    
if __name__ == '__main__':
    warnings.filterwarnings('ignore')
    config = get_config()
    train_model(config)